# Detecting Propaganda in News Articles Using Large Language Models

This notebook contains all of the code used to generate the findings for the `Detecting Propaganda in News Articles Using Large Language Models` paper. 

This can be summarised into the following categories of work:
* Processing of original datasets (which are not included in the repository; you can reach out to me on GitHub to obtain them)
* Generating responses for the articles using the OpenAI gpt-3.5-turbo model
* Processing the responses
* Analysis of the processed responses

In [1]:
# Required libraries
import csv
import openai
import tiktoken
import collections
import pickle
import glob
import os
import re


In [2]:
# Setup configuration
KEY_PATH = "data/openai_key.txt"
MODEL = "gpt-3.5-turbo"
ENC = tiktoken.encoding_for_model(MODEL)
API_KEY = open(KEY_PATH).readline()

openai.api_key = API_KEY


# Importing the pre-built GPT response datasets
For analysis of the results that were already generated by the model (see below).

In [3]:
# Load the data structures instead of creating them from scratch (saves credits and time)
with open("data/first10_article_outputs.pickle", "rb") as first_10:
    responses = pickle.load(first_10)
    
with open("data/semeval2020_gpt_outputs.pickle", "rb") as semeval2020:
    semeval_gpt = pickle.load(semeval2020)

In [43]:
# Analyse data
"""
for article_id, output in responses.items():
    print(article_id)
    print(output)
"""

# Article content
print(semeval_content["article695108099"])
# GPT response
print(semeval_gpt["article695108099"])
# Human label(s) from the dataset
print(semeval["article695108099"])

    

Receipt Shows Paddock Had Another Guest in His Room Before Shooting

This report was originally published by Paul Joseph Watson at Infowars
An image posted on Facebook by a man who claims he served Stephen Paddock room service shows that the gunman was with a guest during his time at the Mandalay Bay hotel four days before the massacre took place.
The receipt shows an employee named Antonio served two guests an “IRA_MB,” which means “In Room Dining, Mandalay Bay.”
The table number is “32135” – which equates to Paddock’s room number (135) on the 32nd floor.
Paddock ordered a burger, a bagel with cream cheese, a potato soup, one bottle of water and two Pepsis, suggesting that there was another person eating with him in the room.
The receipt shows the number “2” next to the word “Guests”.
The date on the receipt also says September 27, whereas authorities assert that Paddock checked in the day after on September 28.
Paddock’s guest could not have been his girlfriend Marilou Danley because

In [ ]:
for k, v in semeval_gpt.items():
    print(k, v)

In [5]:
# Process RT dataset and add contents to filtered 2D array

DATA_PATH = "data/news_articles.csv"
# Set a cut-off point to ensure enough tokens are available
CUTOFF = 3000
articles = []

with open(DATA_PATH, "r", encoding="utf-8") as d:
    for row in csv.reader(d, delimiter=",", quotechar='"'):
            if len(ENC.encode(row[4])) <= CUTOFF:
                articles.append(row)
# Remove header
articles.pop(0)

print(f"Total articles in data set: {len(articles)} / 3702")

Total articles in data set: 3652 / 3702


In [ ]:
# Example article content
print(articles[0][4])


In [38]:
# Process the SemEval-2020 dataset

LABEL_MAP = {"Name_Calling": "Name calling",
            "Repetition": "Repetition",
            "Slogans": "Slogans",
            "Appeal_to_fear-prejudice": "Appeal to fear",
            "Doubt": "Doubt",
            "Exaggeration": "Exaggeration / minimization",
            "Flag-Waving": "Flag-Waving",
            "Loaded_Language": "Loaded Language",
            "Reductio_ad_hitlerum": "Reduction ad hitlerum",
            "Bandwagon": "Bandwagon",
            "Causal_Oversimplification": "Causal oversimplification",
            "Minimisation": "Exaggeration / minimization",
            "Appeal_to_Authority": "Appeal to authority",
            "Black-and-White_Fallacy": "Black & white fallacy",
            "Thought-terminating_Cliches": "Thought terminating clichés",
            "Red_Herring": "Red herring",
            "Straw_Men": "Straw men",
            "Whataboutism": "Whataboutism",
            "Labeling": "Name calling"}

LAP = "data/SemEval-2020_Task_11/articles/*.txt"
LA = "data/SemEval-2020_Task_11/labels/"
SUFFIX = ".task-flc-tc.labels"

semeval = collections.defaultdict(list)
semeval_content = {}

# Build two data structures - semeval (mapping of article_id : labels) & semeval_content (mapping of article_id : article_content)
for article in glob.glob(LAP):
    with open(article, "r", encoding="utf-8") as f:
        article_content = f.read()
        article_name = os.path.basename(article).split('.')[0]
        # Text content is limited to 3000 characters due to token limits
        if len(article_content) < 3000:
            # Get the labels for each article, requires some formatting
            semeval_content[article_name] = article_content
            label = f"{LA}{article_name}{SUFFIX}"
            with open(label, "r", encoding="utf-8") as l:
                labels = l.read()
                if len(labels) > 0:
                    lab_list = set()
                    for i in labels.split("\t"):
                        # Ignore article ID and label positiong
                        if not re.search(r"\d+", i):
                            [lab_list.add(LABEL_MAP[j]) if "," in i else lab_list.add(LABEL_MAP[i]) for j in i.split(",")]
                    semeval[article_name].append(lab_list)
                else:
                    semeval[article_name].append("None")

print(f"Total articles in data set: {len(semeval_content)} / 446")


Total articles in data set: 138 / 446


# Building SemEval-2020 GPT responses from scratch

In [80]:
# === PLEASE NOTE ===
# If following the same methodology, this process will consume approximately 0.3 usd and could take 30+ mins.

semeval_gpt = {}
for k, v in semeval_content.items():
    article_prompt = f"""
The following are a list of propaganda techniques and their definitions:

Name calling - Attack an object/subject of the propaganda with an insulting label.
Repetition - Repeat the same message over and over.
Slogans - Use a brief and memorable phrase.
Appeal to fear - Support an idea by instilling fear against other alternatives.
Doubt - Questioning the credibility of someone/something.
Exaggeration / minimization - Exaggerate or minimize something.
Flag-Waving - Appeal to patriotism or identity.
Loaded Language - Appeal to emotions or stereotypes.
Reduction ad hitlerum - Disapprove an idea suggesting it is popular with groups hated by the audience.
Bandwagon - Appeal to the popularity of an idea.
Causal oversimplification - Assume a simple cause for a complex event.
Obfuscation, intentional vagueness - Use deliberately unclear and obscure expressions to confuse the audience.
Appeal to authority - Use authority’s support as evidence.
Black & white fallacy - Present only two options among many.
Thought terminating clichés - Phrases that discourage critical thought and meaningful discussions.
Red herring - Introduce irrelevant material to distract.
Straw men - Refute argument that was not presented.
Whataboutism - Charging an opponent with hypocrisy.

For each of the techniques and according to its definition, answer with a yes or no if the technique is being used in the following text and with an example from the text if present. This should take the exact form of: "Propaganda technique - Yes or No - Explanation".

{v}

Lastly, give a final verdict on whether the text is propaganda stating a percentage likelihood on the text being propaganda followed by a detailed explanation. This should take the form of: "Verdict - Number% - Explanation".
    """
    try:
        completion = openai.ChatCompletion.create(
            model=MODEL,
            messages=[
                {"role": "user", "content": article_prompt}
            ],
            temperature=0
        )
        output = completion.choices[0].message.content
        print(k)
        print(output)
        print()
        semeval_gpt[k] = output
    except:
        # Best way around this is to retry request later and insert response into semeval_gpt manually
        # If you need to hack something together, the cell output is your friend
        print(f"ERROR - ARTICLE {k} FAILED TO GENERATE RESPONSE")

with open("data/semeval2020_gpt_outputs.pickle", "wb") as w:
    pickle.dump(semeval_gpt, w, protocol=pickle.HIGHEST_PROTOCOL)
        
        

article730269378
Name calling - No
Repetition - No
Slogans - No
Appeal to fear - No
Doubt - Yes - The author questions the credibility of law enforcement by implying they were slow to release Quentin Lamar Smith's name.
Exaggeration / minimization - Yes - The author minimizes the severity of Quentin Lamar Smith's actions by referring to him as simply a "convicted felon" and not mentioning the fact that he allegedly killed two police officers.
Flag-Waving - No
Loaded Language - No
Reduction ad hitlerum - No
Bandwagon - No
Causal oversimplification - No
Obfuscation, intentional vagueness - Yes - The author intentionally obscures the race of Quentin Lamar Smith's wife and makes assumptions based on little evidence.
Appeal to authority - No
Black & white fallacy - No
Thought terminating clichés - No
Red herring - No
Straw men - No
Whataboutism - No

Verdict - 30% - While the text does not use many overt propaganda techniques, it does contain some elements of doubt and intentional vagueness

# Building RT responses from scratch
Currently just 10 articles...

In [120]:
# Send a prompt and receive output for the first 10 articles
# Not necessary if .pickle file present
articles_outputs = {}

for c, article in enumerate(articles):
    if c <= 10:
        article_prompt = f"""
The following are a list of propaganda techniques and their definitions:
                        
Name calling - Attack an object/subject of the propaganda with an insulting label.
Repetition - Repeat the same message over and over.
Slogans - Use a brief and memorable phrase.
Appeal to fear - Support an idea by instilling fear against other alternatives.
Doubt - Questioning the credibility of someone/something.
Exaggeration / minimization - Exaggerate or minimize something.
Flag-Waving - Appeal to patriotism or identity.
Loaded Language - Appeal to emotions or stereotypes.
Reduction ad hitlerum - Disapprove an idea suggesting it is popular with groups hated by the audience.
Bandwagon - Appeal to the popularity of an idea.
Causal oversimplification - Assume a simple cause for a complex event.
Obfuscation, intentional vagueness - Use deliberately unclear and obscure expressions to confuse the audience.
Appeal to authority - Use authority’s support as evidence.
Black & white fallacy - Present only two options among many.
Thought terminating clichés - Phrases that discourage critical thought and meaningful discussions.
Red herring - Introduce irrelevant material to distract.
Straw men - Refute argument that was not presented.
Whataboutism - Charging an opponent with hypocrisy.

For each of the techniques and according to its definition, answer with a yes or no if the technique is being used in the following text and with an example from the text if present. This should take the exact form of: "Propaganda technique - Yes or No - Explanation".

{article[4]}

Lastly, give a final verdict on whether the text is propaganda stating a percentage likelihood on the text being propaganda followed by a detailed explanation. This should take the form of: "Verdict - Number% - Explanation".
"""
        completion = openai.ChatCompletion.create(
            model=MODEL,
            messages=[
                {"role": "user", "content": article_prompt}
            ],
            temperature=0
        )
        output = completion.choices[0].message.content
        print(output)
        articles_outputs[article[0]] = output
    

Name calling - No
Repetition - No
Slogans - No
Appeal to fear - Yes - The article suggests that Russia has plans to commit human rights violations and abuses in Ukraine, which could lead to "widespread human suffering."
Doubt - No
Exaggeration / minimization - No
Flag-Waving - No
Loaded Language - Yes - The article uses phrases such as "human rights catastrophe," "targeted killings," and "lethal measures" to appeal to emotions and create a sense of urgency.
Reduction ad hitlerum - No
Bandwagon - No
Casual oversimplification - No
Obfuscation, intentional vagueness - No
Appeal to authority - Yes - The article cites US Ambassador to the UN Bathsheba Crocker and UN High Commissioner for Human Rights Michelle Bachelet as sources of information.
Black & white fallacy - No
Thought terminating clichés - No
Red herring - No
Straw men - No
Whataboutism - No

Verdict - 30% - While the article does contain some propaganda techniques, such as an appeal to fear and loaded language, the majority of t

In [12]:
# Structure the formatted output into a list of dictionaries
# Example: {8: {'Appeal to fear': 'Yes', 'Explanation': 'The text mentions the fear of a Ukrainian invasion and the escalation of the situation in the region.'}}

responses = collections.defaultdict(list)

def output_to_dict_no_verdict(article_id, output):
    for i in output.split("\n"):
        if "Verdict" in i:
            x = i.split(" - ")
            responses[article_id].append(f"Verdict: {x[1]}")
        else:
            if i != "":
                x = i.split(" - ")
                if len(x) < 3:
                    pass
                else:
                    responses[article_id].append(x[0])

def output_to_dict(article_id, output):
    for i in output.split("\n"):
        if "Verdict" in i:
            x = i.split(" - ")
            responses[article_id].append({"Verdict": x[1], "Explanation": x[2]})
        else:
            if i != "":
                x = i.split(" - ")
                if len(x) < 3:
                    pass
                else:
                    responses[article_id].append({x[0]: x[1], "Explanation": x[2]})

# Convert the output into the structured data using the helper function
for k, v in semeval_gpt.items():
    output_to_dict_no_verdict(k, v)



In [42]:
# Calculate similarity between the SemEval-2020 and gpt-3.5-turbo labels
# 25.12%
similarity = 0
for k, v in responses.items():
    x = set(v)
    y = semeval[k][0]
    z = x.intersection(y)
    pp = 0
    if len(y) and len(z):
         pp = (len(z) / len(y)) * 100
    similarity += pp
    
    print(f"{k} {x} {y} {z} {len(z)} / {len(y)} {pp}%")
    
print(f"{(similarity / len(semeval)):.2f}")

print(semeval_gpt["article999000565"])
print(semeval_content["article999000565"])

# Doesn't get quotes - article999000565

article111111111 {'Appeal to fear', 'Appeal to authority'} {'Doubt', 'Appeal to fear', 'Appeal to authority', 'Repetition'} {'Appeal to fear', 'Appeal to authority'} 2 / 4 50.0%
article111111115 {'Bandwagon', 'Repetition'} {'Loaded Language', 'Appeal to authority'} set() 0 / 2 0%
article111111117 {'Doubt', 'Appeal to authority'} {'Causal oversimplification'} set() 0 / 1 0%
article111111121 {'Doubt', 'Appeal to authority'} None set() 0 / 4 0%
article111111134 {'Doubt'} {'Exaggeration / minimization', 'Repetition', 'Name calling'} set() 0 / 3 0%
article111111136 {'Appeal to fear', 'Whataboutism'} {'Flag-Waving', 'Loaded Language'} set() 0 / 2 0%
article694811415 {'Doubt'} {'Black & white fallacy', 'Bandwagon', 'Reduction ad hitlerum', 'Thought terminating clichés', 'Appeal to authority', 'Doubt'} {'Doubt'} 1 / 6 16.666666666666664%
article695108099 {'Doubt'} None set() 0 / 4 0%
article697063039 {'Appeal to authority', 'Repetition'} {'Exaggeration / minimization', 'Name calling', 'Thought

In [ ]:
for k, v in responses.items():
    for i in set(v):
        print(k, i)


In [15]:
print(len(responses.items()))

138


In [ ]:
# Save the RT test data structure
with open("data/first10_article_outputs.pickle", "wb") as w:
    pickle.dump(responses, w, protocol=pickle.HIGHEST_PROTOCOL)


# Example iteration using a single article from the dataset
This takes `article111111111` (formatted to remove new lines) from the labelled `SemEval-2020` dataset with the prompt template to generate an output from the model.

In [4]:
# OpenAI API Example
# article111111111 (formatted to remove new lines)
article_prompt = f"""
The following are a list of propaganda techniques and their definitions:

Name calling - Attack an object/subject of the propaganda with an insulting label.
Repetition - Repeat the same message over and over.
Slogans - Use a brief and memorable phrase.
Appeal to fear - Support an idea by instilling fear against other alternatives.
Doubt - Questioning the credibility of someone/something.
Exaggeration / minimization - Exaggerate or minimize something.
Flag-Waving - Appeal to patriotism or identity.
Loaded Language - Appeal to emotions or stereotypes.
Reduction ad hitlerum - Disapprove an idea suggesting it is popular with groups hated by the audience.
Bandwagon - Appeal to the popularity of an idea.
Causal oversimplification - Assume a simple cause for a complex event.
Obfuscation, intentional vagueness - Use deliberately unclear and obscure expressions to confuse the audience.
Appeal to authority - Use authority’s support as evidence.
Black & white fallacy - Present only two options among many.
Thought terminating clichés - Phrases that discourage critical thought and meaningful discussions.
Red herring - Introduce irrelevant material to distract.
Straw men - Refute argument that was not presented.
Whataboutism - Charging an opponent with hypocrisy.

For each of the techniques and according to its definition, answer with a yes or no if the technique is being used in the following text and with an example from the text if present. This should take the exact form of: "Propaganda technique - Yes or No - Explanation".

Next plague outbreak in Madagascar could be 'stronger': WHO Geneva - The World Health Organisation chief on Wednesday said a deadly plague epidemic appeared to have been brought under control in Madagascar, but warned the next outbreak would likely be stronger.
"The next transmission could be more pronounced or stronger," WHO Director-General Tedros Adhanom Ghebreyesus told reporters in Geneva, insisting that "the issue is serious."
An outbreak of both bubonic plague, which is spread by infected rats via flea bites, and pneumonic plague, spread person to person, has killed more than 200 people in the Indian Ocean island nation since August.
Madagascar has suffered bubonic plague outbreaks almost every year since 1980, often caused by rats fleeing forest fires.
The disease tends to make a comeback each hot rainy season, from September to April.
On average, between 300 and 600 infections are recorded every year among a population approaching 25 million people, according to a UN estimate.
But Tedros voiced alarm that "plague in Madagascar behaved in a very, very different way this year."
Cases sprang up far earlier than usual and, instead of being confined to the countryside, the disease infiltrated towns.
The authorities recorded more than 2 000 cases, and Tedros said Wednesday the death toll stood at 207.
He also pointed to the presence of the pneumonic version, which spreads more easily and is more virulent, in the latest outbreak.
He praised the rapid response from WHO and Madagascar authorities that helped bring the outbreak under control, but warned that the danger was not over.
The larger-than-usual outbreak had helped spread the bacteria that causes the plague more widely.
This along with poor sanitation and vector control on Madagascar meant that "when (the plague) comes again it starts from more stock, and the magnitude in the next transmission could be higher than the one that we saw," Tedros said.
"That means that Madagascar could be affected more, and not only that, it could even spill over into neighbouring countries and beyond," he warned.
Complicating vector control is the fact that the fleas that carry the Yersinia pestis bacteria that causes the plague have proven to be widely resistant to chemicals and insecticides.
"That's a dangerous combination," Tedros said.

Lastly, give a final verdict on whether the text is propaganda stating a percentage likelihood on the text being propaganda followed by a detailed explanation. This should take the form of: "Verdict - Number% - Explanation".
"""

completion = openai.ChatCompletion.create(
    model=MODEL,
    messages=[
        {"role": "user", "content": article_prompt}
    ],
    temperature=0
)

output = completion.choices[0].message.content
print(output)

Name calling - No
Repetition - Yes - The article repeats the information about the outbreak and the warning of a stronger future outbreak multiple times throughout the text.
Slogans - No
Appeal to fear - Yes - The article warns of a potentially stronger and more widespread outbreak in the future, instilling fear in the reader.
Doubt - No
Exaggeration / minimization - No
Flag-Waving - No
Loaded Language - No
Reduction ad hitlerum - No
Bandwagon - No
Casual oversimplification - No
Obfuscation, intentional vagueness - No
Appeal to authority - Yes - The article quotes the WHO Director-General as an authority on the subject.
Black & white fallacy - No
Thought terminating clichés - No
Red herring - No
Straw men - No
Whataboutism - No

Verdict - 20% - While the article does use some propaganda techniques, such as repetition and appeal to fear, they are not used in a manipulative or deceptive way. The article is primarily focused on reporting factual information about the outbreak and the warn

# Token usage
Here are some simple examples of making token count estimations using actual examples from the datasets. For some ideas on how to comprehensively estimate the token usage, see `utilities/OpenAI_Token_Calculator.py`.

In [173]:
# Get required tokens of example prompt
# Max tokens for prompt and response cannot exceed 4096 tokens
# Prompt without article text comes to 303 tokens

EXAMPLE_PROMPT = f"""
The following are a list of propaganda techniques and their definitions:

Name calling - Attack an object/subject of the propaganda with an insulting label.
Repetition - Repeat the same message over and over.
Slogans - Use a brief and memorable phrase.
Appeal to fear - Support an idea by instilling fear against other alternatives.
Doubt - Questioning the credibility of someone/something.
Exaggeration / minimization - Exaggerate or minimize something.
Flag-Waving - Appeal to patriotism or identity.
Loaded Language - Appeal to emotions or stereotypes.
Reduction ad hitlerum - Disapprove an idea suggesting it is popular with groups hated by the audience.
Bandwagon - Appeal to the popularity of an idea.
Causal oversimplification - Assume a simple cause for a complex event.
Obfuscation, intentional vagueness - Use deliberately unclear and obscure expressions to confuse the audience.
Appeal to authority - Use authority’s support as evidence.
Black & white fallacy - Present only two options among many.
Thought terminating clichés - Phrases that discourage critical thought and meaningful discussions.
Red herring - Introduce irrelevant material to distract.
Straw men - Refute argument that was not presented.
Whataboutism - Charging an opponent with hypocrisy.

For each of the techniques and according to its definition, answer with a yes or no if the technique is being used in the following text and with an example from the text if present. This should take the exact form of: "Propaganda technique - Yes or No - Explanation".

Next plague outbreak in Madagascar could be 'stronger': WHO Geneva - The World Health Organisation chief on Wednesday said a deadly plague epidemic appeared to have been brought under control in Madagascar, but warned the next outbreak would likely be stronger.
"The next transmission could be more pronounced or stronger," WHO Director-General Tedros Adhanom Ghebreyesus told reporters in Geneva, insisting that "the issue is serious."
An outbreak of both bubonic plague, which is spread by infected rats via flea bites, and pneumonic plague, spread person to person, has killed more than 200 people in the Indian Ocean island nation since August.
Madagascar has suffered bubonic plague outbreaks almost every year since 1980, often caused by rats fleeing forest fires.
The disease tends to make a comeback each hot rainy season, from September to April.
On average, between 300 and 600 infections are recorded every year among a population approaching 25 million people, according to a UN estimate.
But Tedros voiced alarm that "plague in Madagascar behaved in a very, very different way this year."
Cases sprang up far earlier than usual and, instead of being confined to the countryside, the disease infiltrated towns.
The authorities recorded more than 2 000 cases, and Tedros said Wednesday the death toll stood at 207.
He also pointed to the presence of the pneumonic version, which spreads more easily and is more virulent, in the latest outbreak.
He praised the rapid response from WHO and Madagascar authorities that helped bring the outbreak under control, but warned that the danger was not over.
The larger-than-usual outbreak had helped spread the bacteria that causes the plague more widely.
This along with poor sanitation and vector control on Madagascar meant that "when (the plague) comes again it starts from more stock, and the magnitude in the next transmission could be higher than the one that we saw," Tedros said.
"That means that Madagascar could be affected more, and not only that, it could even spill over into neighbouring countries and beyond," he warned.
Complicating vector control is the fact that the fleas that carry the Yersinia pestis bacteria that causes the plague have proven to be widely resistant to chemicals and insecticides.
"That's a dangerous combination," Tedros said.

Lastly, give a final verdict on whether the text is propaganda stating a percentage likelihood on the text being propaganda followed by a detailed explanation. This should take the form of: "Verdict - Number% - Explanation".
"""
prompt_tokens = len(ENC.encode(EXAMPLE_PROMPT))


In [174]:
# Example response comes to 602 credits
EXAMPLE_RESPONSE = """
Name calling - No
Repetition - Yes - The article repeats the information about the outbreak and the warning of a stronger future outbreak multiple times throughout the text.
Slogans - No
Appeal to fear - Yes - The article warns of a potentially stronger and more widespread outbreak in the future, instilling fear in the reader.
Doubt - No
Exaggeration / minimization - No
Flag-Waving - No
Loaded Language - No
Reduction ad hitlerum - No
Bandwagon - No
Casual oversimplification - No
Obfuscation, intentional vagueness - No
Appeal to authority - Yes - The article quotes the WHO Director-General as an authority on the subject.
Black & white fallacy - No
Thought terminating clichés - No
Red herring - No
Straw men - No
Whataboutism - No
Verdict - 20% - While the article does use some propaganda techniques, such as repetition and appeal to fear, they are not used in a manipulative or deceptive way.
The article is primarily focused on reporting factual information about the outbreak and the warnings from the WHO."""
resp_tokens = len(ENC.encode(EXAMPLE_RESPONSE))
print(f"Total tokens required: {prompt_tokens + resp_tokens}")


Total tokens required: 1052


# Notes
* Quotes don't work - consumes mass amount of tokens causing broken behaviour.
* The higher the complexity of the output instruction, the more inconsistent the results are. For example, asking to list only the propaganda techniques that are present with a yes / no + quote + explanation causes it to be far too liberal with identifying techniques.
* C